In [1]:
import tensorflow as tf
import cv2
import sys
import numpy as np

C:\Users\prave\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('f', '', 'kernel')

tf.app.flags.DEFINE_string('train_dir', '.\\train_gray.tfrecords',
                       "Directory where train file is present ")

tf.app.flags.DEFINE_string('test_dir', '.\\test_gray.tfrecords',
                       "Directory where test file is present ")

tf.app.flags.DEFINE_string('model_dir', '.\\model\\',
                       "Directory where model will be saved ")

tf.app.flags.DEFINE_string('export_model_dir', '.\\export_model\\',
                       "Directory where model will be saved for serving ")

tf.app.flags.DEFINE_integer('max_steps', 23,
                        """Number of batches to run.""")

tf.app.flags.DEFINE_integer('Btach_size', 25,
                        "batch size")
tf.app.flags.DEFINE_integer('buffer_size', 50,
                        "buffer size")

tf.app.flags.DEFINE_integer('num_classes', 5,
                        "no of classes in dataset")

tf.app.flags.DEFINE_float('learning_rate', 1e-4,
                        "initial learning rate")



In [3]:
def parser(record):
    keys_to_features = {
        "image_raw": tf.FixedLenFeature([], tf.string),
        "label":     tf.FixedLenFeature([], tf.int64)
    }
    parsed = tf.parse_single_example(record, keys_to_features)
    image = tf.decode_raw(parsed["image_raw"], tf.float64)
    #image = tf.cast(image, tf.float64)
    label = tf.cast(parsed["label"], tf.int64)
    return image, label

In [4]:
def input_fn(filenames, train, batch_size=FLAGS.Btach_size, buffer_size=FLAGS.buffer_size):
    dataset = tf.data.TFRecordDataset(filenames=filenames)
    dataset = dataset.map(parser)
    
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
        num_repeat = 2
    else:
        num_repeat = 1
    dataset = dataset.repeat(num_repeat)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()

    images_batch, labels_batch = iterator.get_next()
    x = {'image': images_batch}
    y = labels_batch
    return x, y

In [5]:
with tf.Session() as sess:
    x, y =sess.run(input_fn(filenames=FLAGS.train_dir, train=False))

print(x["image"].shape)
print(y)

(25, 62500)
[4 4 3 0 4 2 4 3 1 2 2 3 4 0 4 0 2 2 2 0 0 3 3 2 0]


In [6]:
def train_input_fn():
    return input_fn(filenames=[FLAGS.train_dir], train=True)

def val_input_fn():
    return input_fn(filenames=[FLAGS.test_dir],train=False)

In [7]:
def model_fn(features, labels, mode, params):
    num_classes = FLAGS.num_classes
    net = features["image"]
    net = tf.identity(net, name="input_tensor")

    net = tf.reshape(net, [-1, 250, 250, 1])
    
    tf.summary.image("my_image", net)

    net = tf.identity(net, name="input_tensor_after")

    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=32, kernel_size=3,
                           padding='valid', activation=tf.nn.relu)

    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=32, kernel_size=3,
                           padding='valid', activation=tf.nn.relu)

    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)
    
    net = tf.layers.dropout(net, rate=0.6, training=(mode == tf.estimator.ModeKeys.TRAIN))

    

    net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=1, padding = "valid")

    net = tf.contrib.layers.flatten(net)

    # net = tf.layers.dense(inputs=net, name='layer_fc1',units=1024, activation=tf.nn.relu)
    
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=128, activation=tf.nn.relu)

    net = tf.layers.dropout(net, rate=0.6, training=(mode == tf.estimator.ModeKeys.TRAIN))

    net = tf.layers.dense(inputs=net, name='layer_fc_3',units=num_classes)

    logits = net

    y_pred = tf.nn.softmax(logits=logits)

    y_pred = tf.identity(y_pred, name="output_pred")

    y_pred_cls = tf.argmax(y_pred, axis=1)

    y_pred_cls = tf.identity(y_pred_cls, name="output_cls")

    if mode == tf.estimator.ModeKeys.PREDICT:
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions = y_pred_cls,
                                          export_outputs = {'predictions': tf.estimator.export.PredictOutput(y_pred_cls)})
        
    else:
        
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)
        loss = tf.reduce_mean(cross_entropy)

        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())
        metrics = {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)

    return spec

In [8]:
def serving_input_fn():
    receiver_tensor = {'image': tf.placeholder(shape=[None, 250, 250, 1], dtype=tf.float64)}
    return tf.estimator.export.ServingInputReceiver(receiver_tensor, receiver_tensor)

In [9]:
def main(args):
    config=tf.estimator.RunConfig(
        save_checkpoints_steps=30, 
        save_summary_steps=30,
        log_step_count_steps  = 20,
        keep_checkpoint_max = 1)
    model = tf.estimator.Estimator(model_fn=model_fn,
                               params={"learning_rate": FLAGS.learning_rate},
                               model_dir=FLAGS.model_dir,
                               config=config)
    print("---------------------")
    print("****traning started*******")
    model.train(input_fn=train_input_fn)
    print("-------------------")
    print("----traning ended---")
    
    export_dir = FLAGS.export_model_dir
    model.export_savedmodel(export_dir, serving_input_fn)
    print("-----------------------")
    print("*********saving the model for serving******")
    
    pre = model.evaluate(input_fn=val_input_fn)
    print(pre) 

In [10]:
if __name__ == '__main__':
    tf.app.run(main=main)

INFO:tensorflow:Using config: {'_model_dir': '.\\model5\\', '_tf_random_seed': None, '_save_summary_steps': 30, '_save_checkpoints_steps': 30, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 20, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001EC5B9EA320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
---------------------
****traning started*******
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into .\model5\model.ckpt.
INFO:tens

SystemExit: 

C:\Users\prave\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
